In [4]:
# https://www.irscalculators.com/tax-calculator

# self employement taxable amount changes federal taxable income, but not FICA taxes
# Schedule SE line 13
# self employed deduction (Schedule SE line 13)
# 199a deduction

# Taxable income and Federal Taxable Income are different

In [5]:
# Default Constants
# LATER convert these to json files or something

FILING_STATUS = "single"

# https://www.fidelity.com/learning-center/smart-money/standard-deduction
STANDARD_DEDUCTIONS = {
    "single":                               15000,
    "married_filing_jointly":               30000,
    "married_filing_separately":            15000,
    "head_of_household":                    22500,
    "single_over_65":                       16950,
    "married_filing_jointly_one_over_65":   31550,
    "married_filing_jointly_both_over_65":  33100,
    "married_filing_separately_over_65":    16950,
    "head_of_household_over_65":            24450

}

STANDARD_DEDUCTION = STANDARD_DEDUCTIONS[FILING_STATUS]
SOCIAL_SECURITY_WAGE_BASE = 168600

# Perminant Constants
PROVISIONAL_INCOME_THRESHOLDS = {
        "single":                    (25000, 34000),
        "head_of_household":         (25000, 34000),
        "married_filing_jointly":    (32000, 44000),
        "married_filing_separately": (0, 0)
}

# INCOME INPUTS
INCOME_INPUTS = {
    "wages":            0,
    "self_employment":  0,
    "unearned_income":  0,
    "capital_gains":    0,
    "social_security":  150000
}

DEDUCTION_INPUTS = {
    "dependents":           0, 
    "standard_deduction":   STANDARD_DEDUCTION,
    "health & retirement":  0
}

In [ ]:
federal_brackets_2025 = {
    "single": [
        (11925, 0.10),
        (48475, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250525, 0.32),
        (626350, 0.35),
        (float('inf'), 0.37)
    ],
    "married_joint": [
        (23850, 0.10),
        (96950, 0.12),
        (206700, 0.22),
        (394600, 0.24),
        (501050, 0.32),
        (751600, 0.35),
        (float('inf'), 0.37)
    ],
    "head": [
        (17000, 0.10),
        (64850, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250500, 0.32),
        (626350, 0.35),
        (float('inf'), 0.37)
    ],
    "married_separate": [
        (11925, 0.10),
        (48475, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250525, 0.32),
        (375800, 0.35),
        (float('inf'), 0.37)
    ]
}

In [7]:
def calculate_taxable_ss(ss_benefits, agi, filing_status):
    # Calculate Provisional Income
    provisional_income = agi + 0.5 * ss_benefits

    # IRS Thresholds
    thresholds = {
        'single': (25000, 34000),
        'head': (25000, 34000),
        'married_joint': (32000, 44000),
        'married_separate': (0, 0)  # Assumes lived with spouse
    }

    if filing_status not in thresholds:
        raise ValueError("Invalid filing status. Use: 'single', 'head', 'married_joint', or 'married_separate'.")

    base, upper = thresholds[filing_status]

    # Calculate taxable portion
    if provisional_income <= base:
        taxable_ss = 0
    elif provisional_income <= upper:
        taxable_ss = 0.5 * (provisional_income - base)
    else:
        taxable_ss = min(
            0.85 * ss_benefits,
            0.85 * (provisional_income - upper) + 0.5 * (upper - base)
        )

    return round(taxable_ss, 2)

In [ ]:
# INCOMPLETE : Figure out the deductions
# Sources seem to be inconsistent 
def taxable_self_employment_income(net_self_employment_income, social_security_wage_base=168600):
    # 92.35% of net SE income is subject to SE tax
    se_taxable_income = net_self_employment_income * 0.9235

    # Social Security tax applies only up to wage base
    ss_taxable = min(se_taxable_income, social_security_wage_base)

    # Medicare tax applies to all SE taxable income
    medicare_taxable = se_taxable_income

    return ss_taxable, medicare_taxable

taxable_self_employment_income(3464845678)

(2770.5, 2770.5)

In [9]:
# Calculate taxable income
def federal_taxable_income(income, deductions, filing_status):
   # AGI has above the line deductions included, but these are not factored in.  
   adjusted_gross_income = sum(income.values()) - income['social_security']
   income['social_security'] = calculate_taxable_ss(income['social_security'], adjusted_gross_income, filing_status)
#  income['self_employement'] = ...
   total_income = sum(income.values())
   total_deductions = sum(deductions.values())
   return max(total_income - total_deductions, 0)
   

federal_taxable_income(INCOME_INPUTS, DEDUCTION_INPUTS, FILING_STATUS)

24350.0